<h2>Coleta de dados - Mestrado</h2>

Esse código é responsável por coletar ligantes de um determinado alvo a partir do API do ChEMBL.

Começamos importando as bibliotecas:

In [156]:
import math
from pathlib import Path
from zipfile import ZipFile
from tempfile import TemporaryDirectory

import numpy as np
import pandas as pd
from rdkit.Chem import PandasTools
from chembl_webresource_client.new_client import new_client
from tqdm.auto import tqdm

Na sequência, setamos o caminho para salvar os arquivos

In [157]:
HERE = Path(_dh[-1])
DATA = HERE / "data"

#checamos a versão do python utilizada para rodar os códigos
from platform import python_version
python_version()

'3.10.4'

In [158]:
#Criando os resource objects para o acesso ao API
targets_api = new_client.target
compounds_api = new_client.molecule
bioactivities_api = new_client.activity

In [159]:
type(targets_api)
uniprot_id = "P13922"

Primeiro, pegamos a <b>UniProt ID</b> do alvo de interesse, nesse caso vamos começar utilizando a DHFR-TS que é a "P13922"
Coletamos numa variável (uniprot_id) e coletamos os alvos na variável "targets"

In [107]:
# Get target information from ChEMBL but restrict it to specified values only
targets = targets_api.get(target_components__accession=uniprot_id).only(
    "target_chembl_id", "organism", "pref_name", "target_type"
)
print(f'O tipo do alvo é "{type(targets)}"') #essa resposta devera ser chembl_webresource_client.query_set.QuerySet

O tipo do alvo é "<class 'chembl_webresource_client.query_set.QuerySet'>"


Os resultados do <i>query</i> ficam armazenados na variavel "targets" e os resultados não são reconhecidos até usarmos o pandas para buscá-los.

In [108]:
targets = pd.DataFrame.from_records(targets)
targets

,organism,pref_name,target_chembl_id,target_type
0,Plasmodium falciparum K1,Dihydrofolate reductase,CHEMBL1939,SINGLE PROTEIN
1,Plasmodium falciparum K1,Dihydrofolate reductase,CHEMBL1939,SINGLE PROTEIN


Selecionamos apenas a primeira linha da tabela acima. Na verdade, nesse caso tanto faz pois o target_chembl_id é o mesmo para ambas as linhas.

In [109]:
target = targets.iloc[1]

In [110]:
chembl_id = target.target_chembl_id #retiramos o chembl_id para filtrarmos as moleculas e atividades biologicas relacionadas
print(f"O alvo ChEMBL ID é {chembl_id}") 
# checamos o ID do alvo no ChEMBL

O alvo ChEMBL ID é CHEMBL1939


---
<h3>Baixando as atividades biológicas:</h3>

Em sequência, baixamos as atividades biológicas e filtramos apenas os valores de Ki presentes;

In [134]:
bioactivities = bioactivities_api.filter(target_chembl_id=chembl_id, type="Ki", relation="=", assay_type="B").only(
    "activity_id", #o tipo de atividade
    "assay_chembl_id",#o numero do ensaio relacionado
    "assay_description",#descricao do ensaio relacionado
    "assay_type", #tipo de ensaio
    "molecule_chembl_id", #id da molecula
    "type", #tipo
    "standard_units", #unidades
    "relation", #relacao
    "standard_value",#valor padronizado
    "target_chembl_id", #id chembl do alvo
    "target_organism", #organismo alvo
)
print(f"Length and type of bioactivities object: {len(bioactivities)}, {type(bioactivities)}")

Length and type of bioactivities object: 716, <class 'chembl_webresource_client.query_set.QuerySet'>


Como o objeto é uma lista encadeada, selecionamos, por fim apenas o dataframe contido no primeiro elemento.

In [135]:
print(f"Length and type of first element: {len(bioactivities[0])}, {type(bioactivities[0])}")
bioactivities[0] #como o objeto é uma lista encadeada, selecionamos apenas o dataframe contido no primeiro elemento
;

Length and type of first element: 13, <class 'dict'>


''

<h3>Transformação em dataframe:</h3>

Para trabalharmos da melhor maneira possível com os dados, assim como no R, transformamos a lista encadeada em um dataframe usando a funcao <b>pd.Dataframe.from_records</b>

In [136]:
bioactivities_df = pd.DataFrame.from_records(bioactivities)
print(f"DataFrame shape: {bioactivities_df.shape}")
bioactivities_df.head()

DataFrame shape: (717, 13)


,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,standard_units,standard_value,target_chembl_id,target_organism,type,units,value
0,185071,CHEMBL811761,Binding affinity was evaluated as inhibition o...,B,CHEMBL2364573,=,nM,4.7,CHEMBL1939,Plasmodium falciparum K1,Ki,nM,4.7
1,185071,CHEMBL811761,Binding affinity was evaluated as inhibition o...,B,CHEMBL2364573,=,nM,4.7,CHEMBL1939,Plasmodium falciparum K1,Ki,nM,4.7
2,185072,CHEMBL811759,Binding affinity was evaluated as inhibition o...,B,CHEMBL2364573,=,nM,14.5,CHEMBL1939,Plasmodium falciparum K1,Ki,nM,14.5
3,187525,CHEMBL821379,Inhibitory activity against wild-type dihydrof...,B,CHEMBL324775,=,nM,3.1,CHEMBL1939,Plasmodium falciparum K1,Ki,nM,3.1
4,187527,CHEMBL668412,Inhibitory activity against double mutant dihy...,B,CHEMBL324775,=,nM,103.5,CHEMBL1939,Plasmodium falciparum K1,Ki,nM,103.5


Verificamos o tipo de cada variavel e transformamos os valores de atividade biologica em "float" para evitarmos que seja truncado ou arredondado e comprometa de alguma forma a nossa análise

In [137]:
#como parte de boas praticas, verificamos o tipo de cada variavel e transformamos a variavel em float para que não seja truncado ou arredondado
bioactivities_df = bioactivities_df.astype({"standard_value": "float64"})

bioactivities_df.dtypes

activity_id             int64
assay_chembl_id        object
assay_description      object
assay_type             object
molecule_chembl_id     object
relation               object
standard_units         object
standard_value        float64
target_chembl_id       object
target_organism        object
type                   object
units                  object
value                  object
dtype: object

Também como forma de boas práticas de modelagem, retiramos todos os valores tidos como "NA" no banco de moléculas, uma vez que isso inviabilizará a obtenção de dados e/ou modelos mais pra frente. Nesse caso em específico, vemos que não há nenhuma variável com dados faltantes.

In [138]:
bioactivities_df.dropna(axis=0, how="any", inplace=True)
print(f"DataFrame shape: {bioactivities_df.shape}")

DataFrame shape: (717, 13)


É importante conferir se existe alguma outra unidade de medida presente no banco de dados obtido. Nesse caso não observamos nenhum outro tipo de unidade de medida biologica a nao ser nM.

In [139]:
print(f"Tipos de unidades nos dados baixados: {bioactivities_df['standard_units'].unique()}")

print(    f"Numero de dados sem valores na faixa do nanomolar nM:\
    {bioactivities_df[bioactivities_df['standard_units'] != 'nM'].shape[0]}")

Tipos de unidades nos dados baixados: ['nM']
Numero de dados sem valores na faixa do nanomolar nM:    0


Observamos que não há nenhuma atividade biológica medida de outra forma que não o nanomolar. Mesmo que nesse caso não se aplique, é importante sempre filtrar o banco de dados para uma mesma unidade de medida.

In [140]:
bioactivities_df = bioactivities_df[bioactivities_df["standard_units"] == "nM"]
print(f"Units after filtering: {bioactivities_df['standard_units'].unique()}")
print(f"DataFrame shape: {bioactivities_df.shape}")

Units after filtering: ['nM']
DataFrame shape: (717, 13)


---
<h3>Removendo estruturas redundantes</h3>

Agora chegamos em um ponto bastante crucial para o desenvolvimento dos modelos: a quantidade de moléculas. É fato que temos 717 medições experimentais, mas na verdade temos a mesma quantidade de moléculas? Ou foram realizados várias medições biológicas para um número menor de estruturas. A resposta se dá abaixo com o "drop_duplicates". Mantemos as que aparecem primeiro

In [142]:
bioactivities_df.drop_duplicates("molecule_chembl_id", keep="first", inplace=True)
print(f"DataFrame shape: {bioactivities_df.shape}")

DataFrame shape: (145, 13)


Pois é, vemos que na verdade das mais de 700 atividades biológicas medidas, possuimos apenas 145 estruturas diferentes. A ideia agora é juntar as estruturas para a espécie <i>falciparum</i> e <i>vivax</i>! (Acho que para outra espécie temos cerca de ~30 estruturas)

In [153]:
bioactivities_df.reset_index(drop=True, inplace=True)
bioactivities_df.head()

,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,standard_units,standard_value,target_chembl_id,target_organism,type,units,value
0,185071,CHEMBL811761,Binding affinity was evaluated as inhibition o...,B,CHEMBL2364573,=,nM,4.7,CHEMBL1939,Plasmodium falciparum K1,Ki,nM,4.7
1,187525,CHEMBL821379,Inhibitory activity against wild-type dihydrof...,B,CHEMBL324775,=,nM,3.1,CHEMBL1939,Plasmodium falciparum K1,Ki,nM,3.1
2,188709,CHEMBL821379,Inhibitory activity against wild-type dihydrof...,B,CHEMBL416373,=,nM,0.4,CHEMBL1939,Plasmodium falciparum K1,Ki,nM,0.4
3,189896,CHEMBL821379,Inhibitory activity against wild-type dihydrof...,B,CHEMBL291931,=,nM,7.2,CHEMBL1939,Plasmodium falciparum K1,Ki,nM,7.2
4,189912,CHEMBL821379,Inhibitory activity against wild-type dihydrof...,B,CHEMBL119188,=,nM,0.6,CHEMBL1939,Plasmodium falciparum K1,Ki,nM,0.6
